In [1]:
%run 00_display_tables.ipynb

Successfully connected to py_brew database.
+-------------+-------------------+---------------+----------------+
| customer_id |   customer_name   | customer_city | customer_state |
+-------------+-------------------+---------------+----------------+
|     100     | Jennifer Gonzalez |   Smithland   |    Arkansas    |
+-------------+-------------------+---------------+----------------+
+---------+---------------------+------------+-------------+-------+------+
| date_id |        tmstp        |    date    | day_of_week | month | year |
+---------+---------------------+------------+-------------+-------+------+
|    1    | 2020-07-04 21:23:10 | 2020-07-04 |      4      |   7   | 2020 |
+---------+---------------------+------------+-------------+-------+------+
+------------------+------------------+---------------------+--------------+---------------+
|    product_id    | product_category | product_subcategory | product_name | product_price |
+------------------+------------------+------

In [2]:
%%sql

DROP PROCEDURE IF EXISTS GetProductCategorySales;

CREATE PROCEDURE GetProductCategorySales(
    IN p_category VARCHAR(100)
)
BEGIN
    SELECT 
        product_category,
        DATE_FORMAT(date, '%Y-%b') AS sale_month,
        SUM(sales_amount) AS total_sales,
        LAG(SUM(sales_amount)) OVER (PARTITION BY product_category ORDER BY DATE_FORMAT(date, '%Y-%b')) AS previous_month_sales,
        ROUND(((SUM(sales_amount) - LAG(SUM(sales_amount)) OVER (PARTITION BY product_category ORDER BY DATE_FORMAT(date, '%Y-%b'))) / SUM(sales_amount) * 100), 2) AS growth_rate
    FROM 
        fact_sales
        JOIN dim_date ON fact_sales.date_id = dim_date.date_id
        JOIN dim_product ON fact_sales.product_id = dim_product.product_id
    WHERE 
        product_category = p_category
        AND date >= DATE_SUB(CURDATE(), INTERVAL 12 MONTH)
    GROUP BY product_category, DATE_FORMAT(date, '%Y-%b');
END;


[]

In [3]:
%%sql 
-- not supported
-- CALL GetProductCategorySales('Retail')

[]